In [1]:

import pandas as pd
import numpy as np


#!pip install geopandas
#!pip install geopy
#!pip install folium

import folium
import geopandas
import geopy
from folium.plugins.marker_cluster import MarkerCluster
import folium.plugins as plugins
from geopy.geocoders import Nominatim
from geopy.extra.rate_limiter import RateLimiter

In [3]:
# Test
locator = Nominatim(user_agent='my_request')
location = locator.geocode('højlundsparken 71, fastrup 8355 solbjerg')

print('Latitude = {}, Longitude = {}'.format(location.latitude, location.longitude))



Latitude = 56.038386, Longitude = 10.069766


In [8]:

df1 = pd.read_csv('addresses.csv')

df1["ADDRESS"] = df1['Address1'].astype(str) + ", " + df1['Address3'].astype(str) + "," + df1['Address4'].astype(str) + "," + df1['Address5'].astype(str) + ", " + "Sweden"
df1.head()

# 1 - conveneint function to delay between geocoding calls
geocode = RateLimiter(locator.geocode, min_delay_seconds=1)
# 2- - create location column
df1['location'] = df1['ADDRESS'].apply(geocode)
# 3 - create longitude, laatitude and altitude from location column (returns tuple)
df1['point'] = df1['location'].apply(lambda loc: tuple(loc.point) if loc else None)
# 4 - split point column into latitude, longitude and altitude columns
df1[['latitude', 'longitude', 'altitude']] = pd.DataFrame(df1['point'].tolist(), index=df.index).df1.dropna()
df1.head(30)

In [9]:
#df1 = df1.drop(['ADDRESS', 'Address3', 'Address4', 'Address5','Telefon', 'ADDRESS', 'location', 'point','Namn'], axis=1)


,Unnamed: 0,Typ,Nr,Namn,Address1,Address3,Address4,Address5,Telefon,ADDRESS,location,point,latitude,longitude,altitude
0,0,Butik,102,Fältöversten,Karlaplan 13,115 20,STOCKHOLM,Stockholms län,08/662 22 89,"Karlaplan 13, 115 20,STOCKHOLM,Stockholms län,...","(13, Karlaplan, Östermalm, Östermalms stadsdel...","(59.3388767, 18.0908655, 0.0)",59.338877,18.090865,0.0
2,2,Butik,106,Garnisonen,Karlavägen 100 A,115 26,STOCKHOLM,Stockholms län,08/662 64 85,"Karlavägen 100 A, 115 26,STOCKHOLM,Stockholms ...","(Karlavägen, Östermalm, Östermalms stadsdelsom...","(59.3356727, 18.0991791, 0.0)",59.335673,18.099179,0.0
4,4,Butik,113,Sergel,Drottninggatan 45,111 21,STOCKHOLM,Stockholms län,08/21 47 44,"Drottninggatan 45, 111 21,STOCKHOLM,Stockholms...","(Drottninggatan, Klara, Norrmalm, Norrmalms st...","(59.3336131, 18.0616743, 0.0)",59.333613,18.061674,0.0
5,5,Butik,114,PK-Huset,Norrlandsgatan 3,111 47,STOCKHOLM,Stockholms län,08/796 98 10,"Norrlandsgatan 3, 111 47,STOCKHOLM,Stockholms ...","(3, Norrlandsgatan, Klara, Norrmalm, Norrmalms...","(59.3331769, 18.0712808, 0.0)",59.333177,18.071281,0.0
7,7,Butik,132,Marieberg,Rålambsvägen 7-9,112 59,STOCKHOLM,Stockholms län,08/13 30 95,"Rålambsvägen 7-9, 112 59,STOCKHOLM,Stockholms ...","(7-9, Rålambsvägen, Marieberg, Kungsholmens st...","(59.3279896, 18.0174112, 0.0)",59.327990,18.017411,0.0
12,13,Butik,140,Torsplan,Solnavägen 2b,113 65,STOCKHOLM,Stockholms län,08/31 73 49,"Solnavägen 2b, 113 65,STOCKHOLM,Stockholms län...","(2B, Solnavägen, Röda bergen, Vasastaden, Norr...","(59.3464104, 18.0332565, 0.0)",59.346410,18.033257,0.0
13,14,Butik,143,Odenplan,Odengatan 58,113 22,STOCKHOLM,Stockholms län,08/32 35 29,"Odengatan 58, 113 22,STOCKHOLM,Stockholms län,...","(58, Odengatan, Sibirien, Vasastaden, Norrmalm...","(59.3434945, 18.0528016, 0.0)",59.343494,18.052802,0.0
14,15,Butik,144,Jarlaplan,Birger Jarlsgatan 84,114 20,STOCKHOLM,Stockholms län,08/24 31 15,"Birger Jarlsgatan 84, 114 20,STOCKHOLM,Stockho...","(84, Birger Jarlsgatan, Lärkstaden, Östermalm,...","(59.3446986, 18.063395, 0.0)",59.344699,18.063395,0.0
19,24,Butik,166,Söderhallarna,Medborgarplatsen 3,118 26,STOCKHOLM,Stockholms län,08/641 86 75,"Medborgarplatsen 3, 118 26,STOCKHOLM,Stockholm...","(3, Medborgarplatsen, Södermalm, Södermalms st...","(59.3144846, 18.0712026, 0.0)",59.314485,18.071203,0.0
23,30,Butik,173,Hammarby Sjöstad,Lugnets Allé 26-28,120 66,STOCKHOLM,Stockholms län,08/640 00 44,"Lugnets Allé 26-28, 120 66,STOCKHOLM,Stockholm...","(26-28, Lugnets Allé, Södra Hammarbyhamnen, Sö...","(59.3023089, 18.1034195, 0.0)",59.302309,18.103420,0.0


In [10]:
map1 = folium.Map(
    location=[59.338315,18.089960],
    tiles='CartoDB dark_matter',
    zoom_start=12,
)
df1.apply(lambda row:folium.CircleMarker(location=[row["latitude"], row["longitude"]]).add_to(map1), axis=1)
map1